# TSA Chapter 5: GARCH Model Comparison

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_comparison/TSA_ch5_comparison.ipynb)

Information criteria comparison across GARCH model variants on S&P 500.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
import yfinance as yf

# Download S&P 500 data
sp = yf.download('^GSPC', start='2000-01-01', end='2025-12-31', progress=False)
if isinstance(sp.columns, pd.MultiIndex):
    sp.columns = sp.columns.get_level_values(0)
sp500_returns = (sp['Close'].pct_change() * 100).dropna()
sp500_returns = pd.Series(sp500_returns.values, index=sp500_returns.index, name='returns')
print(f"S&P 500: {len(sp500_returns)} observations")

In [ ]:
from arch import arch_model

specs = {
    'GARCH-N': dict(vol='Garch', p=1, q=1, dist='normal'),
    'GARCH-t': dict(vol='Garch', p=1, q=1, dist='t'),
    'GJR-t':   dict(vol='Garch', p=1, o=1, q=1, dist='t'),
    'EGARCH-t': dict(vol='EGARCH', p=1, q=1, dist='t'),
}
aic_vals = []; bic_vals = []; vols = {}
for name, kw in specs.items():
    m = arch_model(sp500_returns, **kw)
    r = m.fit(disp='off')
    aic_vals.append(r.aic); bic_vals.append(r.bic)
    vols[name] = r.conditional_volatility
    print(f"{name}: AIC={r.aic:.1f}, BIC={r.bic:.1f}")

names = list(specs.keys()); x = np.arange(len(names)); w = 0.35

fig, axes = plt.subplots(1, 2, figsize=(13, 5))
axes[0].bar(x - w/2, aic_vals, w, label='AIC', color=BLUE, alpha=0.8)
axes[0].bar(x + w/2, bic_vals, w, label='BIC', color=RED, alpha=0.8)
axes[0].set_ylabel('Criterion Value'); axes[0].set_title('Information Criteria', fontweight='bold')
axes[0].set_xticks(x); axes[0].set_xticklabels(names, rotation=15, ha='right')
min_idx = np.argmin(aic_vals)
axes[0].annotate('Best', xy=(x[min_idx] - w/2, aic_vals[min_idx]),
                 xytext=(x[min_idx] - w/2, aic_vals[min_idx] - 500),
                 fontsize=9, color=GREEN, fontweight='bold',
                 arrowprops=dict(arrowstyle='->', color=GREEN))
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), frameon=False, ncol=2)

last_n = 500; colors_v = [BLUE, RED, GREEN, ORANGE]
for (name, vol), col in zip(vols.items(), colors_v):
    axes[1].plot(sp500_returns.index[-last_n:], vol[-last_n:], lw=0.8, color=col, label=name, alpha=0.8)
axes[1].set_xlabel('Date'); axes[1].set_ylabel('Volatility (%)')
axes[1].set_title('Estimated Volatility - Last 500 Days', fontweight='bold')
axes[1].xaxis.set_major_locator(mdates.YearLocator())
axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), frameon=False, ncol=4)

fig.tight_layout(); fig.subplots_adjust(bottom=0.18)
save_chart(fig, 'garch_comparison'); plt.show()